In [ ]:
import {config} from "dotenv"

config({
  path: ".env.local",
})


In [2]:
import { TavilySearch } from "@langchain/tavily"
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio"
import { tool } from "@langchain/core/tools"
import { z } from "zod"

const tavilyTool = new TavilySearch()

// @ts-ignore
const scrapeWebpage = tool(
  async (input) => {
    const loader = new CheerioWebBaseLoader(input.url)
    const docs = await loader.load()
    const formattedDocs = docs.map(
      (doc) =>
        `<Document name="${doc.metadata?.title}">\n${doc.pageContent}\n</Document>`
    )
    return formattedDocs.join("\n\n")
  },
  {
    name: "scrape_webpage",
    description: "Scrape the contents of a webpage.",
    schema: z.object({
      url: z.string(),
    }),
  }
)


In [ ]:
import "tsx"
import { createCanvas } from "canvas"
import * as d3 from "d3"
import * as tslab from "tslab"
import * as fs from "fs/promises"
import * as path from "path"
import { tool } from "@langchain/core/tools"
import { z } from "zod"

const WORKING_DIRECTORY = "./temp"
await fs.mkdir(WORKING_DIRECTORY, { recursive: true })

//@ts-ignore
const createOutlineTool = tool(
  async ({ points, file_name }) => {
    const filePath = path.join(WORKING_DIRECTORY, file_name)
    const data = (points as any)
      .map((point, index) => `${index + 1}.${point}\n`)
      .join("\n")
    await fs.writeFile(filePath, data)
    return `Outline saved to ${file_name}`
  },
  {
    name: "create_outline",
    description: "Create and save an outline",
    schema: z.object({
      points: z
        .array(z.string())
        .nonempty("List of main points or sections must not be empty."),
      file_name: z.string(),
    }),
  }
)
